In [37]:
!pip install python-dotenv
!pip install huggingface_hub

import requests
import os

from huggingface_hub import InferenceClient
from dotenv import load_dotenv


In [5]:


load_dotenv()

HF_API_TOKEN = os.getenv("HUGGINGFACE_TOKEN_API")


In [7]:

API_URL_I2T = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-base"
headers_i2t = {"Authorization": f"Bearer {HF_API_TOKEN}"}

def img2text(filename):

    with open(filename, "rb") as f:
        data = f.read()
        
    response = requests.post(API_URL_I2T , headers=headers_i2t , data=data)
    return response.json()[0]['generated_text']



In [14]:

client = InferenceClient( "HuggingFaceH4/zephyr-7b-beta", token = HF_API_TOKEN )

def text2story(img_text):

    r = client.chat_completion(
	messages=[
            {"role": "system",
             "content": "As an experienced short story writer, write story title and then create a meaningful story influenced by provided words. Ensure stories conclude positively within 100 words. Remember the story must end within 100 words"},
            {"role": "user", "content": f"Here is input set of words: {img_text}"} ] )

    return r.choices[0].message.content

In [22]:
API_URL_T2S = "https://api-inference.huggingface.co/models/espnet/kan-bayashi_ljspeech_vits"
headers_t2s = {"Authorization": f"Bearer {HF_API_TOKEN}"}

def story2speech(text):
    
    payloads = {'inputs': text}

    response = requests.post(API_URL_T2S, headers=headers_t2s, json=payloads)
    
    with open('audio_story.mp3', 'wb') as file:
        file.write(response.content)


In [28]:
import IPython

In [34]:

def app(file_name):
    output_i2t = img2text(file_name)
    output_t2s = text2story(output_i2t)
    story2speech(output_t2s)

    return (output_i2t,output_t2s)
    
  
    



In [35]:
img_text , img_story = app("animals.jpg")

In [36]:
print(img_text)
print("------------")
print(img_story)
print("------------")
IPython.display.Audio("audio_story.mp3")


a large elephant standing in a field with many animals
------------
"The Gathering"

In the heart of the savannah, a large elephant stood somberly in a vast field. The sun was just beginning to rise above the horizon, casting a warm, orange glow over the landscape. The elephant's trunk swayed gently in the pre-dawn breeze, and its ears twitched, listening intently for any sound in the distance.

As the day broke and the animals of the savannah began to
------------
